# Fit energies and forces (full cycle)

In [38]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")


In [42]:
from fande.data import FandeDataModule
from fande.utils.find_atomic_groups import find_atomic_groups

fdm = FandeDataModule()

In [99]:
trajectory_forces = traj_295_2000K[0:5000:10]
trajectory_forces = trajectory_forces[0::10].copy()
trajectory_energy = traj_295[0:5000:10] + traj_355[0:5000:10] + traj_295_2000K[0:5000:10] + traj_355_2000K[0:5000:10] + traj_295_2000K_forced[0:5000:10] + traj_355_2000K_forced[0:5000:10]
trajectory_energy = trajectory_energy[:].copy()

print(len(trajectory_forces), len(trajectory_energy))

50 3000


In [100]:
soap_params = dict(soap_type="PowerSpectrum",
        interaction_cutoff=3.0,
        max_radial=3,
        max_angular=3,
        gaussian_sigma_constant=0.3,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=0.1, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=True, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True, # for energies gradients are ignored
        expansion_by_species_method='structure wise'
        )
##FOR NOW USE THE SAME SOAP PARAMETERS FOR ENERGY AND FORCES! (that makes sense if you're modeling the MD)

sample_snapshot = trajectory_forces[0].copy()
fdm = FandeDataModule()
atomic_groups = find_atomic_groups(sample_snapshot)
train_centers_positions = sum(atomic_groups, []) #list(range(len(atoms)))
train_derivatives_positions = sum(atomic_groups, [])#list(range(len(atoms)))
fdm.atomic_groups_sample_snapshot = sample_snapshot.copy()
fdm.atomic_groups = atomic_groups

total_forces_samples_per_group = [3000] * len(atomic_groups)
high_forces_samples_per_group = [0] * len(atomic_groups)

Total number of found groups: 14
Checking if all atoms are covered:  True


In [101]:
dataloader_energy, dataloaders_forces = fdm.dataloaders_from_trajectory(
                                                                trajectory_energy,
                                                                trajectory_forces,
                                                                # energies = None,
                                                                # forces = None,
                                                                atomic_groups = atomic_groups,
                                                                centers_positions = train_centers_positions,
                                                                derivatives_positions = train_derivatives_positions,
                                                                energy_soap_hypers = soap_params,
                                                                forces_soap_hypers = soap_params,
                                                                total_forces_samples_per_group = total_forces_samples_per_group,
                                                                high_force_samples_per_group = high_forces_samples_per_group,
                                                                )

Total length of traj is 3000
Total number of batches 3000
Calculating invariants on trajectory with librascal...


100%|██████████| 3000/3000 [01:22<00:00, 36.44it/s]


invariants for energy fitting calculated


100%|██████████| 50/50 [00:22<00:00,  2.21it/s]


invariants for forces fitting calculated
Taking ALL 1800 samples for group 0
Dataloader for group 0 created
Number of samples in dataloader: 1800
Dataloader for group 1 created
Number of samples in dataloader: 3000
Dataloader for group 2 created
Number of samples in dataloader: 3000
Taking ALL 1800 samples for group 3
Dataloader for group 3 created
Number of samples in dataloader: 1800
Taking ALL 1800 samples for group 4
Dataloader for group 4 created
Number of samples in dataloader: 1800
Dataloader for group 5 created
Number of samples in dataloader: 3000
Dataloader for group 6 created
Number of samples in dataloader: 3000
Dataloader for group 7 created
Number of samples in dataloader: 3000
Dataloader for group 8 created
Number of samples in dataloader: 3000
Dataloader for group 9 created
Number of samples in dataloader: 3000
Taking ALL 1800 samples for group 10
Dataloader for group 10 created
Number of samples in dataloader: 1800
Dataloader for group 11 created
Number of samples in d

In [102]:
# Making energy model

from fande.models import EnergyModel

hparams = {
        'dtype' : 'float32',
        'device' : 'gpu',
        'energy_model_hparams' : {
                'num_epochs' : 200,
                'learning_rate' : 0.1,
        }
        }
       
Energy_model = EnergyModel(
        dataloader_energy,
        hparams=hparams)

Energy_model.fit()

/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dlbox2/anaconda3/envs/fande/lib/python3.10/sit ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type                       | Params
----------------------------------------------------------
0 | likelihood | GaussianLikelihood         | 1     
1 | model      | ExactGPModelEnergy         | 542   
2 | mll        | ExactMarginalLogLikelihood | 542   
----------------------------------------------------------
542       Trainable params
0         Non-trainable params
542       Total params
0.


RawEnergyModel initialized
Training energy model


/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


In [ ]:
# %%time
# # Fitting forces

# from fande.models import ModelForces, GroupModelForces


# n_steps_list = [200] * len(atomic_groups)
# lr_list = [0.1] * len(atomic_groups)

# models_hparams = []
# for i in range(len(atomic_groups)):
#         model_hparams = {
#         'atomic_group' : atomic_groups[i],
#         'dtype' : hparams['dtype'],
#         'device' : hparams['device'],
#         'num_epochs' : n_steps_list[i],
#         'learning_rate' : lr_list[i],
#         'soap_dim' : dataloaders_forces[i].dataset[0][0].shape[-1],
#         'soap_params' : soap_params,
#         }
#         models_hparams.append(model_hparams)

# hparams['per_model_hparams'] = models_hparams # access per_model_hparams by model.model_id
# gpu_id = 0


# models_forces = []
# for i in range(len(atomic_groups)):
#         model = ModelForces(
#         train_x = dataloaders_forces[i].dataset[:][0],
#         train_y = dataloaders_forces[i].dataset[:][1],
#         atomic_group = atomic_groups[i],
#         hparams = hparams,
#         id=i)
#         models_forces.append(model)
        
# AG_force_model = GroupModelForces(
#         models= models_forces,
#         train_data_loaders = dataloaders_forces,
#         hparams=hparams, 
#         gpu_id=gpu_id)

# AG_force_model.fit()

In [103]:
from fande.predict import FandePredictor
from fande.ase import FandeCalc

# Energy_model = None
AG_force_model = None
predictor = FandePredictor(
        fdm,
        AG_force_model,
        Energy_model,
        hparams,
        soap_params
        )

fande_calc = FandeCalc(predictor)

In [138]:
# fande_calc.save_predictor("fande_predictor.pth")

Saving predictor requires humongous amount of memory! Spare some dozens of GBs!


In [105]:
# move to cpu
device = torch.device('cpu')
fande_calc.predictor.energy_model_device = device

fande_calc.predictor.energy_model = fande_calc.predictor.energy_model.to(device)
fande_calc.predictor.energy_model.model.likelihood = fande_calc.predictor.energy_model.model.likelihood.to(device)
fande_calc.predictor.energy_model.model.model = fande_calc.predictor.energy_model.model.model.to(device)
fande_calc.predictor.energy_model.model.model.train()
fande_calc.predictor.energy_model.model.likelihood.train()
fande_calc.predictor.energy_model.model.model.eval()
fande_calc.predictor.energy_model.model.likelihood.eval()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [107]:
from tqdm import tqdm
test_traj = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="1100:1110")
test_traj = test_traj.copy()

real_energies = [s.get_potential_energy() for s in test_traj]
predicted_energies = []
for i in tqdm(range(len(test_traj))):
        test_traj[i].calc = fande_calc
        # predicted_energies.append( test_traj[i].get_potential_energy() )
        # print(test_traj[i].get_potential_energy() )
        # test_traj[i].get_forces()
        print(test_traj[i].get_potential_energy() )
        # test_traj[i].get_forces()

 10%|█         | 1/10 [00:00<00:04,  2.07it/s]

Time for invariants (call from forces):  473.7539291381836
Energy model device:  cpu
Energy model summary: 
Time invariants:  0.10585784912109375
Time prediction:  4.501581192016602
Time moving on device:  0.5564689636230469
Time total:  6.011486053466797
[-3080.8547]


 20%|██        | 2/10 [00:00<00:03,  2.07it/s]

Time for invariants (call from forces):  475.0244617462158
Energy model device:  cpu
Energy model summary: 
Time invariants:  0.10657310485839844
Time prediction:  4.194736480712891
Time moving on device:  0.5598068237304688
Time total:  5.685091018676758
[-3080.9036]


 30%|███       | 3/10 [00:01<00:03,  2.07it/s]

...

Time for invariants (call from forces):  475.4374027252197
Energy model device:  cpu
Energy model summary: 
Time invariants:  0.1068115234375
Time prediction:  4.139184951782227
Time moving on device:  0.5726814270019531
Time total:  5.6438446044921875
[-3082.104]


 90%|█████████ | 9/10 [00:04<00:00,  2.07it/s]

Time for invariants (call from forces):  476.12619400024414
Energy model device:  cpu
Energy model summary: 
Time invariants:  0.10514259338378906
Time prediction:  4.180908203125
Time moving on device:  0.5552768707275391
Time total:  5.668401718139648
[-3082.1985]


100%|██████████| 10/10 [00:04<00:00,  2.07it/s]

Time for invariants (call from forces):  474.8384952545166
Energy model device:  cpu
Energy model summary: 
Time invariants:  0.10561943054199219
Time prediction:  4.197359085083008
Time moving on device:  0.5555152893066406
Time total:  5.6819915771484375
[-3082.2507]


In [69]:
test_traj[10].calc = fande_calc
# predicted_energies.append( test_traj[i].get_potential_energy() )
# print(test_traj[i].get_potential_energy() )
# test_traj[i].get_forces()
print(test_traj[10].get_potential_energy() )

IndexError: list index out of range

In [106]:
import numpy as np
np.any(True)

True

In [85]:
%%time
# test_traj[0].get_forces()
for i in tqdm(range(len(test_traj))):
        test_traj[i].get_forces()
# test_traj[1].get_potential_energy()

 30%|███       | 6/20 [00:07<00:16,  1.20s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(real_energies, label="real")
plt.plot(predicted_energies, label="predicted")
plt.legend()
plt.show()

In [14]:
atoms = trajectory_energy[51].copy()

atoms.set_calculator(fande_calc)

print(atoms.get_potential_energy())
print(atoms.get_forces())

Atomic group force model is not defined. Cannot predict forces. Returning zeros.
[-3080.6099]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [139]:
fande_calc.save_predictor("prrrrr.pt")

Saving predictor requires humongous amount of memory! Spare some dozens of GBs!


In [143]:
import torch
from fande.ase import FandeCalc

fande_calc = FandeCalc(None)
fande_calc.load_predictor("prrrrr.pt")

In [154]:
atoms = fande_calc.predictor.fdm.traj_train[0]
atoms.set_calculator(fande_calc)
print(atoms.get_potential_energy())
print(atoms.get_forces())

[-3101.314]
[[ 0.00611877  0.02384949 -0.15219116]
 [-0.02478027  0.02558899  0.43151855]
 [-0.03694153  0.41552734 -0.02424622]
 ...
 [-0.11352539 -0.11639404 -0.04455566]
 [-0.00396729 -0.03076172 -0.02807617]
 [ 0.31091309  0.01507568 -0.10125732]]


In [4]:
fande_calc_loaded = FandeCalc(predictor_loaded)

In [6]:
from ase import io
atoms = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="10")

atoms.set_calculator(fande_calc_loaded)

In [1]:
atoms.get_potential_energy()
atoms.get_forces()

NameError: name 'atoms' is not defined

array([[0.00688136, 0.00688136, 0.00688136],
       [0.00688136, 0.00688136, 0.00688136],
       [0.00688136, 0.00688136, 0.00688136],
       ...,
       [0.16884041, 0.16884041, 0.16884041],
       [0.16884041, 0.16884041, 0.16884041],
       [0.16884041, 0.16884041, 0.16884041]])

# see when the benefit appears

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 10, 40_000)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

In [3]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [62]:
dev = torch.device("cpu") 

model = model.to(dev)
likelihood = likelihood.to(dev)
train_x = train_x.to(dev)
train_y = train_y.to(dev)

In [63]:
%%time
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 5


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/5 - Loss: -0.175   lengthscale: 0.133   noise: 0.049
Iter 2/5 - Loss: -0.178   lengthscale: 0.121   noise: 0.044
Iter 3/5 - Loss: -0.178   lengthscale: 0.110   noise: 0.040
Iter 4/5 - Loss: -0.177   lengthscale: 0.101   noise: 0.037
Iter 5/5 - Loss: -0.169   lengthscale: 0.092   noise: 0.035
CPU times: user 30min 57s, sys: 3min 43s, total: 34min 40s
Wall time: 10min 11s


In [68]:
dev = torch.device("cpu") 

model = model.to(dev)
likelihood = likelihood.to(dev)
test_x = torch.rand(1_0).to(dev)

train_x = train_x.to(dev)
train_y = train_y.to(dev)



In [54]:
# Get into evaluation (predictive posterior) mode
model.train()
likelihood.train()
model.eval()
likelihood.eval()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [69]:
%%time

model.eval()
likelihood.eval()
# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))

CPU times: user 12min 20s, sys: 21.5 s, total: 12min 41s
Wall time: 2min 51s


In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(4, 3))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])

In [263]:
import gc
torch.cuda.empty_cache()
gc.collect()

0